###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Defensa Nacional de Colombia

In [43]:
import pandas as pd

###Archivo p72f-qcvk.csv

Contiene información sobre erradicación de cultivos ilícitos que incluye la eliminación física o afectación directa de los cultivos ilegales. Se ejecuta a través del empleo de diferentes modalidades que buscan la disminución de este tipo de plantaciones. La unidad de medida corresponde a hectáreas (ha).

In [44]:
df = pd.read_csv("https://www.datos.gov.co/resource/p72f-qcvk.csv?$limit=500000", dtype={'cod_muni': str})
df.head()

,fecha_hecho,cod_depto,departamento,cod_muni,municipio,tipo_de_cultivo,cantidad,unidad
0,2007-01-01T00:00:00.000,85,CASANARE,85300,SABANALARGA,COCA,1.0,HECTAREA
1,2007-01-02T00:00:00.000,52,NARIÑO,52678,SAMANIEGO,AMAPOLA,4.0,HECTAREA
2,2007-01-03T00:00:00.000,68,SANTANDER,68101,BOLIVAR,COCA,3.0,HECTAREA
3,2007-01-04T00:00:00.000,68,SANTANDER,68190,CIMITARRA,COCA,2.0,HECTAREA
4,2007-01-04T00:00:00.000,15,BOYACA,15531,PAUNA,COCA,1.0,HECTAREA


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143224 entries, 0 to 143223
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   fecha_hecho      143224 non-null  object 
 1   cod_depto        143224 non-null  int64  
 2   departamento     143224 non-null  object 
 3   cod_muni         143224 non-null  object 
 4   municipio        143224 non-null  object 
 5   tipo_de_cultivo  143224 non-null  object 
 6   cantidad         143218 non-null  float64
 7   unidad           143224 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 8.7+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [46]:
relevant_cols = ['cod_muni', 'departamento', 'municipio', 'fecha_hecho', 'tipo_de_cultivo', 'cantidad', 'unidad']
eradication_of_illicit_crops_MinDefensa = df[relevant_cols]
eradication_of_illicit_crops_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143224 entries, 0 to 143223
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   cod_muni         143224 non-null  object 
 1   departamento     143224 non-null  object 
 2   municipio        143224 non-null  object 
 3   fecha_hecho      143224 non-null  object 
 4   tipo_de_cultivo  143224 non-null  object 
 5   cantidad         143218 non-null  float64
 6   unidad           143224 non-null  object 
dtypes: float64(1), object(6)
memory usage: 7.6+ MB


### Convertir columna fecha_hecho a tipo date

In [47]:
# Hacer una copia explícita del DataFrame
eradication_of_illicit_crops_MinDefensa_ = eradication_of_illicit_crops_MinDefensa.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
eradication_of_illicit_crops_MinDefensa_['fecha_hecho'] = pd.to_datetime(eradication_of_illicit_crops_MinDefensa_['fecha_hecho'], errors='coerce')

# Extraer solo el año
eradication_of_illicit_crops_MinDefensa_['fecha_hecho'] = eradication_of_illicit_crops_MinDefensa_['fecha_hecho'].dt.year

In [48]:
eradication_of_illicit_crops_MinDefensa_['fecha_hecho'].unique()

array([2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
       2018, 2019, 2020, 2021, 2022, 2023, 2024], dtype=int32)

### Verificar valores nulos

In [49]:
eradication_of_illicit_crops_MinDefensa_.isnull().sum()

,0
cod_muni,0
departamento,0
municipio,0
fecha_hecho,0
tipo_de_cultivo,0
cantidad,6
unidad,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [50]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio', 'unidad', 'tipo_de_cultivo']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(eradication_of_illicit_crops_MinDefensa_[column].unique())
    print()


Categorías en la columna 'departamento':
['CASANARE' 'NARIÑO' 'SANTANDER' 'BOYACA' 'CALDAS' 'TOLIMA' 'CUNDINAMARCA'
 'ANTIOQUIA' 'META' 'CAQUETA' 'CAUCA' 'VALLE DEL CAUCA' 'CESAR' 'PUTUMAYO'
 'ARAUCA' 'NORTE DE SANTANDER' 'VICHADA' 'BOLIVAR' 'CORDOBA' 'GUAVIARE'
 'HUILA' 'CHOCO' 'LA GUAJIRA' 'MAGDALENA' 'GUAINIA' 'AMAZONAS' 'RISARALDA'
 'VAUPES']

Categorías en la columna 'municipio':
['SABANALARGA' 'SAMANIEGO' 'BOLIVAR' 'CIMITARRA' 'PAUNA' 'VICTORIA'
 'FUNES' 'ROBERTO PAYAN' 'TUQUERRES' 'EL ROSARIO' 'ATACO' 'RICAURTE'
 'CAPARRAPI' 'EL PEÑON' 'PUERTO TRIUNFO' 'VELEZ' 'LEIVA' 'TOPAIPI' 'URIBE'
 'POLICARPA' 'LA UNION' 'YACOPI' 'LA MONTAÑITA' 'CHAPARRAL' 'FLORENCIA'
 'SAN PABLO DE BORBUR' 'TUNUNGUA' 'EL PLAYON' 'DAGUA' 'SANTA MARIA' 'PAEZ'
 'BARBACOAS' 'SAMANA' 'SAN LUIS DE GACENO' 'EL PEÑOL' 'AGUSTIN CODAZZI'
 'OSPINA' 'ORITO' 'SAN ALBERTO' 'PUERTO RICO' 'PUERTO RONDON' 'TAMINANGO'
 'PAILITAS' 'LA ESPERANZA' 'OTANCHE' 'MEDINA' 'BUENAVENTURA'
 'LA PRIMAVERA' 'SIMITI' 'SAN MIGUEL' 'PAIME' 

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [51]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [52]:
# Aplicar la función a todas las columnas categóricas
eradication_of_illicit_crops_MinDefensa_[categorical_col] = eradication_of_illicit_crops_MinDefensa_[categorical_col].astype(str)
for col in categorical_col:
    eradication_of_illicit_crops_MinDefensa_[col] = eradication_of_illicit_crops_MinDefensa_[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "cod_muni" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [53]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_muni" en el df eradication_of_illicit_crops_MinDefensa_

In [54]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
eradication_of_illicit_crops_MinDefensa_['cod_muni'] = eradication_of_illicit_crops_MinDefensa_['cod_muni'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = eradication_of_illicit_crops_MinDefensa_['cod_muni'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

True

In [55]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5]


In [56]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_muni,count,percentage
0,5,143224,1.0


In [57]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = eradication_of_illicit_crops_MinDefensa_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_muni']])
    print()

Muestra de registros con longitud 5:
  cod_muni
0    85300
1    52678
2    68101
3    68190
4    15531



Nota: El 100%  de los codigos que identifican a los municipios en el df eradication_of_illicit_crops_MinDefensa corresponden a string de 5 caracteres, igual a los almacenados en la base de datos del proyecto

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [58]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [59]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [60]:
# Comparar listas de departamento
compare_lists(eradication_of_illicit_crops_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en eradication_of_illicit_crops_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en eradication_of_illicit_crops_MinDefensa_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [61]:
# Comparar listas de códigos
compare_lists(eradication_of_illicit_crops_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en eradication_of_illicit_crops_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en eradication_of_illicit_crops_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [63]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_eradication_of_illicit_crops_MinDefensa = eradication_of_illicit_crops_MinDefensa_.drop(columns=columns_to_drop)

In [65]:
# Adicionar columna para trazabilidad de la fuente
final_eradication_of_illicit_crops_MinDefensa['source_id'] = 34

In [66]:
final_eradication_of_illicit_crops_MinDefensa.columns

Index(['cod_muni', 'fecha_hecho', 'tipo_de_cultivo', 'cantidad', 'unidad',
       'source_id'],
      dtype='object')

In [67]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'cod_muni': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'tipo_de_cultivo': 'type_of_crop',
    'cantidad': 'amount',
    'unidad': 'unit',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_eradication_of_illicit_crops_MinDefensa.rename(columns=translation_map, inplace=True)

In [68]:
#Estructura final del dataset a integrar a la base de datos
final_eradication_of_illicit_crops_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143224 entries, 0 to 143223
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   dane_code         143224 non-null  object 
 1   year_of_incident  143224 non-null  int32  
 2   type_of_crop      143224 non-null  object 
 3   amount            143218 non-null  float64
 4   unit              143224 non-null  object 
 5   source_id         143224 non-null  int64  
dtypes: float64(1), int32(1), int64(1), object(3)
memory usage: 6.0+ MB


## Salvar en archivo csv en el drive

In [69]:
# Guardar en archivos CSV en el drive
final_eradication_of_illicit_crops_MinDefensa.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/eradication_of_illicit_crops_MinDefensa.csv', index=False)